In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer

dummy_reg = DummyRegressor(strategy="mean")

Test_Target = ["SUSPECT_ARRESTED_FLAG","FIREARM_FLAG", "STOP_DURATION_MINUTES", "STOP_DURATION_MINUTES"]
Test1 = pd.read_excel("sqf2022.xlsx", usecols=Test_Target)
Feature_test=["FIREARM_FLAG", "STOP_DURATION_MINUTES", "STOP_DURATION_MINUTES"]



Test1['FIREARM_FLAG'] = Test1['FIREARM_FLAG'].replace('(null)', 'No')
Test1.replace('(null)', float('nan'), inplace=True)

lb = LabelBinarizer()
Test1["SUSPECT_ARRESTED_FLAG"] = lb.fit_transform(Test1["SUSPECT_ARRESTED_FLAG"])
Test1["FIREARM_FLAG"] = lb.fit_transform(Test1["FIREARM_FLAG"])
X = Test1[["SUSPECT_ARRESTED_FLAG"]]
y = Test1[Feature_test]
Test1.to_csv('output.csv', index=False)


train_data,test_data=train_test_split(Test1,test_size=0.2,random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=20)
#dummy_reg.fit(X_train, y_train)
#y_train_predict = dummy_reg.predict(X_train)

#trainerror = mean_squared_error(y_train, y_train_predict, squared=False)

#y_test_pred = dummy_reg.predict(X_test)
#testerror = mean_squared_error(y_test, y_test_pred, squared=False)

#print("Training error:", trainerror)
#print("Test error:", testerror)


In [33]:
model = LinearRegression()
model.fit(train_data[Feature_test], train_data["SUSPECT_ARRESTED_FLAG"])
predictions=model.predict(test_data[Feature_test])
train_predictions=model.predict(train_data[Feature_test])
rmse = mean_squared_error(test_data["SUSPECT_ARRESTED_FLAG"],predictions,squared=False)
print('RMSE: ', rmse)
train_rmse=mean_squared_error(train_data["SUSPECT_ARRESTED_FLAG"],train_predictions,squared=False)
print("Train_RMSE:",train_rmse)

ValueError: could not convert string to float: 'Based on Radio Run'